# Import modules

In [2]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy


# Functions (helpers)

In [3]:
def read_csv(name, info=True, sep=','):
    df = pd.read_csv(f'../data/demographics/{name}.csv', sep=sep)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print('\n')
    print( '*' + '-' * 135 + '*')
    print('* ' + title)
    print( '*' + '-' * 135 + '*')

def print_subsection(title):
    print('\n')
    print('***** ' + title + ' *****')
    print( '*' + '-' * 135 + '*')

In [4]:
# filenames = [
#    'interleaved_incomplete', 'block_complete',
#    'block_complete_simple',
#    'block_complete_mixed',  'block_complete_mixed_2s','block_complete_mixed_2s_amb_heuristic',
#    'block_complete_mixed_2s_amb_final', 'incentivize', 'evoutcome'

# ]
# dem = {}
# for f in filenames:
#     if f == 'block_complete_mixed_2s_amb_final':
#         dem[f] = pd.read_csv('../data/demographics/' + f + '.csv', sep=';')
#     else: dem[f] = read_csv(f)

# # d = dem["block_incomplete"][dem['block_incomplete']['status'] == 'APPROVED']
# # d
# df = {}
# ids = pd.read_csv('../sub_ids.csv')
# ids['prolific_id'] = 'nothing'
# for f in filenames:
#     df = pd.read_csv('../data/csv/' + f + '.csv', header=None)
#     # df.loc[-1] = [str(i) for i in range(len(df.columns))]  # adding a row
#     # df.index = df.index + 1  # shifting index
#     # df.sort_index(inplace=True) 
#     for i in ids[ids.exp_name==f]['id']:
#         try:
#             prolific_id = df[df[1]==i][2].unique()[0]
#         # break
#             ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#         except:
#             try:
#                 prolific_id = df[df[1]==str(i)][2].unique()[0]
#                 ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#             except:
#                 print(i, ' is missing...')
    
# # for i in ids['id']:
#     # df
# ids.to_csv('../sub_ids_with_prolific_id.csv', index=False)


In [9]:
df = pd.read_csv('../data/demographics/included_subjects_without_revoked.csv')

ids = df['prolific_id'].unique()
exps = df['exp_name'].unique()
# df['age'] = np.nan
# df['gender'] = np.nan

for f in exps:
    sep = ','
    # if f == 'block_incomplete':
    #       #get prolific ids from demographics
    #     ddf = pd.read_csv('../data/demographics/block_incomplete.csv')
    #     ddf = ddf[ddf.status == 'APPROVED']
    #     df.loc[df.exp_name=='block_incomplete', 'prolific_id'] = idx
    #     ids = df['prolific_id'].unique()

    if f == 'block_complete_mixed_2s_amb_final':
        sep = ';'
    print(f)
    dem = pd.read_csv('../data/demographics/' + f + '.csv', sep=sep)
    for i in ids:
        try:
            df.loc[df.prolific_id==i, 'gender'] = dem.loc[dem['participant_id'] == i, 'Sex'].values[0]
            df.loc[df.prolific_id==i, 'age'] = dem.loc[dem['participant_id'] == i, 'age'].values[0]
            df.loc[df.prolific_id==i, 'bonus'] = dem.loc[dem['participant_id'] == i, 'bonus'].values[0]

        except:
            print(i, ' is missing...')
            pass
df.to_csv('../data/demographics/included_subjects_with_bonus.csv')

interleaved_incomplete
5c06e9fa0ea1410001a9c28a@email.prolific.ac  is missing...
ttp://app.prolific.co/verify-email/?i=UB0N3LKNNNY1  is missing...
5d3241a6d63753001ad72d82  is missing...
5d10fe33fbbf3d001a56d936  is missing...
5cf0f51eb094d700013e5136  is missing...
5cd45ad839aa1a0001596d4c  is missing...
5bbc0997f1f9ba000141cf0f  is missing...
5bd08b85669ee1000143ddbc  is missing...
5c45b0d5b05faa0001d60dd3  is missing...
5d2a0577c22768001666cdbf  is missing...
5c4bb5a2620ba700010bf02b  is missing...
5cb5e6057f36f00015e8dc0b  is missing...
5d5e66a74cb63d00193fe0ba  is missing...
5cd3e8f3c088820001e9e537  is missing...
5d5b0f01e7a79e0017fc56a1  is missing...
5896f9951a06b50001a7ff12  is missing...
5b213b968ca8150001dd1109  is missing...
5bb36695bfaca5000125901f  is missing...
5d52b4628c48ca0017855e69  is missing...
5b7aa778f9bc170001a9328e  is missing...
5bdcd8f78b0d7e0001228e13  is missing...
5beaf5c1ca3a1a0001823677  is missing...
5cacc9cd0f757800019d0bac  is missing...
5d5faf38887cf

In [44]:


df = pd.read_csv('../data/demographics/included_subjects_without_revoked.csv')
#df.loc[(df['gender'].isna()) | (df['gender']=='CONSENT REVOKED'), 'gender'] = 'DATA_EXPIRED'
#df.to_csv('../data/demographics/included_subjects_without_revoked.csv')
#df = df[(df.exp_num != 9) & (df.exp_num!=10)]
df
#df[df['age'] == 'DATA_EXPIRED'] = np.random.choice('')
d = []
for e in df.exp_num.unique():
    if e == 10: continue
    if e == 9: pass

    #print(f'Exp: {e} | ', sum(df[df.exp_num==e].bonus.isna()))
    gender = df[df.exp_num==e].gender.value_counts()
    age = df[(df.exp_num==e) & (df.age != 'DATA_EXPIRED')]
    exp_name = df[df.exp_num==e].exp_name.values[0]
    without = gender.drop(labels=['Male', 'Female'], inplace=False)
    other_gender = without.sum()
        
    d.append(
        {   'exp_num': e,
            'exp_name': exp_name,
            'age_std': age.age.astype(float).std(),
            'age': age.age.astype(float).mean(),
            'prefer_not': other_gender,
            'M': gender['Male'],
            '%M': (gender['Male']/sum(gender)).round(2),
            'F': gender['Female'],
            '%F': (gender['Female']/sum(gender)).round(2),
            'N': len(df[df.exp_num==e]),

    })
# sum(df.bonus.isna())
#df.bonus.std()
pd.DataFrame(d) 

# new_df.N.sum()
# # new_df.age.mean()
# # new_df.age_std.mean()

# new_df.F.sum()
# new_df.M.sum()
# new_df.prefer_not.sum()
# 307+331+35
# pg.ttest(df.bonus, 2.5, confidence=.975)
# len(df.bonus)-67

,Unnamed: 0.1,Unnamed: 0,id,exp_num,exp_name,prolific_id,age,gender
0,0,0,76422442,1,interleaved_incomplete,5d30c0f46cc6bd0020882925,26.0,Female
1,1,1,192489031,1,interleaved_incomplete,5d3ebe453a0c0100162313b9,31.0,Female
2,2,2,324224518,1,interleaved_incomplete,5d3ebb0d64c213001ad49fb0,24.0,Female
3,3,3,543099212,1,interleaved_incomplete,5cd875edf7470b0016e9ca83,29.0,Female
4,4,4,635786936,1,interleaved_incomplete,5d0bccb311a6b5001588347b,63.0,Male
...,...,...,...,...,...,...,...,...
748,748,748,9049475133,10,evoutcome,5f1e9db721e4f101783fe21c,21,Male
749,749,749,9195332330,10,evoutcome,61119ee3576ae2aa2cc8cc32,19,Female
750,750,750,9391982197,10,evoutcome,5f68f02c41716709ee9be42d,25,Male
751,751,751,9484643174,10,evoutcome,615767861330080eb8cad21c,36,Female


,exp_num,exp_name,age_std,age,prefer_not,M,%M,F,%F,N
0,1,interleaved_incomplete,9.753738,30.628571,7,34,0.45,35,0.46,76
1,2,block_incomplete,9.331194,30.765625,6,26,0.37,39,0.55,71
2,3,block_complete,8.757050,30.812500,4,41,0.49,38,0.46,83
3,4,block_complete_simple,7.346840,28.426829,7,48,0.55,33,0.38,88
4,5,block_complete_mixed,12.436505,32.666667,2,38,0.54,31,0.44,71
5,6,block_complete_mixed_2s,8.974671,28.953846,2,32,0.48,32,0.48,66
6,7,block_complete_mixed_2s_amb_heuristic,10.251463,32.768116,3,32,0.45,36,0.51,71
7,8,block_complete_mixed_2s_amb_final,8.910623,29.028169,2,33,0.45,38,0.52,73
8,9,incentivize,11.215476,30.068493,2,47,0.64,25,0.34,74
